La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



# Problemas 6 y 7

Este notebook contiene la implementación explicada para los dos siguientes problemas:


- Implementa un método, non_accesible, que reciba un vértice, vertex, y
devuelva la lista vértices del grafo que no son accesibles desde vertex. Un
vértice end es no accesible desde vertex si no existe ningún camino de vertex a
end.

- Implementa un método, get_recheable, que reciba un vértice, vertex, y
devuelva la lista todos los vértices para los que existe un camino desde vertex.
a) Implementa una solución basada en bfs.
b) Implementa una solución basada en dfs.


In [7]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        else:
            return str(self.vertex)

In [8]:
from queue import Queue

class Graph:
    def __init__(self, vertices: list, directed: bool = True) -> None:
        """ We use a dictionary to represent the graph
        the dictionary's keys are the vertices
        The value associated for a given key will be the list of their neighbours.
        Initially, the list of neighbours is empty"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []
        self._directed = directed

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))

        if not self._directed:
            # adds to the end of the list of neighbors for end
            self._vertices[end].append(AdjacentVertex(start, weight))

    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result

    
    def bfs(self, start: object) -> list:
        """BFS traversal from start"""
        result = []
        visited = dict.fromkeys(self._vertices.keys(), False)

        q = Queue()
        q.put(start)
        visited[start] = True

        while not q.empty():
            v = q.get()
            result.append(v)
            lst_adj = self.get_adjacent_vertices(v)
            for v in lst_adj:
                if not visited[v]:
                    visited[v] = True
                    q.put(v)

        return result
        
    def dfs(self, start: object) -> list:
        """DFS traversal (based on recursion) from start"""

        result = []
        visited = dict.fromkeys(self._vertices.keys(), False)
        
        self._dfs(start, visited, result)
        return result

    def _dfs(self, start: object, visited: dict, result: list) -> None:
        visited[start] = True
        result.append(start)
        for v in self.get_adjacent_vertices(start):
            if not visited[v]:
                self._dfs(v, visited, result)   

Vamos a extender Graph para que contenga los dos nuevos métodos:

In [9]:
class Graph2(Graph):

    def non_accesible(self, start: object) -> list:
        """returns a list containing those vertices that cannot be accesed from start"""
        # first, we get those vertices that can be accessed from start
        accesibles = self.dfs(start)
        # now we obtain those vertices that are not in the previous list, accesibles
        result = [v for v in self._vertices.keys() if v not in accesibles]    
        return result

    def get_reachable(self, start: object) -> list:
        """returns a list containing those vertices that cannot be accesed from start"""
        # we must simply returns one of the traversal: dfs or bfs from start
        return self.dfs(start)
        # return self.bfs(start)
        



<img src='https://i.stack.imgur.com/OYCLI.png' width='300'>


In [10]:
labels = ['A', 'B', 'C', 'D', 'E', 'F']
g = Graph2(labels, False)

g.add_edge('A','B')
g.add_edge('A','C')
g.add_edge('B','C')
g.add_edge('C','D')

g.add_edge('E','F')
print(g)
for v in g._vertices.keys():
    print("non accesible from {}: {}".format(v, g.non_accesible(v)))
    print("accesible from {}: {}".format(v, g.get_reachable(v)))




A:(B,1)  (C,1)  
B:(A,1)  (C,1)  
C:(A,1)  (B,1)  (D,1)  
D:(C,1)  
E:(F,1)  
F:(E,1)  

non accesible from A: ['E', 'F']
accesible from A: ['A', 'B', 'C', 'D']
non accesible from B: ['E', 'F']
accesible from B: ['B', 'A', 'C', 'D']
non accesible from C: ['E', 'F']
accesible from C: ['C', 'A', 'B', 'D']
non accesible from D: ['E', 'F']
accesible from D: ['D', 'C', 'A', 'B']
non accesible from E: ['A', 'B', 'C', 'D']
accesible from E: ['E', 'F']
non accesible from F: ['A', 'B', 'C', 'D']
accesible from F: ['F', 'E']


<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/Directed_graph%2C_cyclic.svg/900px-Directed_graph%2C_cyclic.svg.png' widht='50'>

In [11]:
labels = ['A', 'B', 'C', 'D', 'E', 'F']
g = Graph2(labels)

g.add_edge('A','B')
g.add_edge('B','C')
g.add_edge('C','E')

g.add_edge('E','D')
g.add_edge('E','F')

g.add_edge('D','B')
print(g)

for v in g._vertices.keys():
    print("non accesible from {}: {}".format(v, g.non_accesible(v)))
    print("accesible from {}: {}".format(v, g.get_reachable(v)))




A:(B,1)  
B:(C,1)  
C:(E,1)  
D:(B,1)  
E:(D,1)  (F,1)  
F:

non accesible from A: []
accesible from A: ['A', 'B', 'C', 'E', 'D', 'F']
non accesible from B: ['A']
accesible from B: ['B', 'C', 'E', 'D', 'F']
non accesible from C: ['A']
accesible from C: ['C', 'E', 'D', 'B', 'F']
non accesible from D: ['A']
accesible from D: ['D', 'B', 'C', 'E', 'F']
non accesible from E: ['A']
accesible from E: ['E', 'D', 'B', 'C', 'F']
non accesible from F: ['A', 'B', 'C', 'D', 'E']
accesible from F: ['F']
